# Crawling Drama
 - 드라마 목록 위키([링크](https://ko.wikipedia.org/wiki/2018%EB%85%84_%EB%8C%80%ED%95%9C%EB%AF%BC%EA%B5%AD%EC%9D%98_%ED%85%94%EB%A0%88%EB%B9%84%EC%A0%84_%EB%93%9C%EB%9D%BC%EB%A7%88_%EB%AA%A9%EB%A1%9D))
<!-- -인스타 / 페이스북 대체 페이지([링크](https://pikdo.net/u/styleandstar/2064045967)) -->

In [1]:
!pip install --force https://github.com/chengs/tqdm/archive/colab.zip

     | 522kB 9.5MB/s
  Running setup.py bdist_wheel for tqdm ... - \ done
  Stored in directory: /tmp/pip-ephem-wheel-cache-tln6hxra/wheels/41/18/ee/d5dd158441b27965855b1bbae03fa2d8a91fe645c01b419896
Successfully built tqdm
spacy 2.0.18 has requirement numpy>=1.15.0, but you'll have numpy 1.14.6 which is incompatible.
pymc3 3.6 has requirement joblib<0.13.0, but you'll have joblib 0.13.0 which is incompatible.
featuretools 0.4.1 has requirement pandas>=0.23.0, but you'll have pandas 0.22.0 which is incompatible.
  Found existing installation: tqdm 4.28.1
    Uninstalling tqdm-4.28.1:
      Successfully uninstalled tqdm-4.28.1


In [2]:
import pandas as pd
import numpy as np
import sys
import requests
import json
from bs4 import BeautifulSoup as bs
import json
from datetime import *
from urllib.request import Request, urlopen
from tqdm import tqdm_notebook as tqdm
from google.colab import drive 
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


## Get Data from Wikipedia

In [0]:
url = 'https://ko.wikipedia.org/wiki/2018%EB%85%84_%EB%8C%80%ED%95%9C%EB%AF%BC%EA%B5%AD%EC%9D%98_%ED%85%94%EB%A0%88%EB%B9%84%EC%A0%84_%EB%93%9C%EB%9D%BC%EB%A7%88_%EB%AA%A9%EB%A1%9D'
res = requests.get(url).text
soup = bs(res, 'lxml')

### Extract data from table on Wikipedia
- data contains **title, link, date **...

In [0]:
def makelist(table):
  url_base = 'https://ko.wikipedia.org'
  result = []
  allrows = table.findAll('tr')
  for row in allrows:
    result.append([])
    allcols = row.findAll('td')
    for col in allcols:
      texts = col.text.replace('《','').replace('》','').replace('\n','')
      links = col.findAll('a')
      if links == []:
         result[-1].append(texts +";")
      else:
        for link in links:
          result[-1].append(texts +";"+ url_base + link['href'])
  return result

def getcol(table):
  result = []
  allrows = table.findAll('th')
  for row in allrows:
    result.append(row.text.replace('\n',''))
  return result


### Make Dataframe
- drop useless data from dataframe 

In [5]:
tables = soup.find_all('table',{'class':'wikitable'})[1:4]
len(tables)
dra_list = []
for table in tables:
  dra_list.extend(makelist(table)[1:])
df_dra = pd.DataFrame(dra_list)

df_dra.head()

,0,1,2,3,4,5,6,7,8,9,10,11
0,조선미인별전;https://ko.wikipedia.org/wiki/%EC%A1%B0...,경민선;,김대현;,2018.01.062018.01.07;,2;,KBS1;,특집;,;,예;,;,;,None
1,문집;,신하은;,이윤정;https://ko.wikipedia.org/wiki/%EC%9D%B4%EC...,2018.01.06;,1;,tvN;,단막(토);,;,예;,;,;,None
2,낫 플레이드(Not Played);,강민수;,유종선;,2018.01.13;,1;,tvN;,단막(토);,;,예;,;,;,None
3,리턴;https://ko.wikipedia.org/wiki/%EB%A6%AC%ED%...,최경미;,주동민이정림;,2018.01.172018.03.22;,34 (17);,SBS;,수목;,;,예;,;,2회 연장;,None
4,우리 집은 맛나 된장 맛나;,윤조;,김상호;https://ko.wikipedia.org/wiki/%EA%B9%80%EC...,2018.01.20;,1;,tvN;,단막(토);,;,예;,;,;,None


In [6]:
df_dra.iloc[35,3] = '2018.04.282018.06.03;'
df_dra = df_dra[[0,3]]
df_dra['Title'] = ''
df_dra['Link'] = ''
df_dra['Start_date'] = ''
df_dra['End_date'] = ''
df_dra['Title'] = df_dra[0].map(lambda x:x.split(';')[0])
df_dra['Link'] = df_dra[0].map(lambda x:x.split(';')[-1])
df_dra['Start_date'] = df_dra[3].map(lambda x:x[:10])
df_dra['End_date'] = df_dra[3].map(lambda x:x[10:].replace(';',''))

df_dra = df_dra[['Title','Link','Start_date','End_date']]
df_dra.head()

,Title,Link,Start_date,End_date
0,조선미인별전,https://ko.wikipedia.org/wiki/%EC%A1%B0%EC%84%...,2018.01.06,2018.01.07
1,문집,,2018.01.06,
2,낫 플레이드(Not Played),,2018.01.13,
3,리턴,https://ko.wikipedia.org/wiki/%EB%A6%AC%ED%84%...,2018.01.17,2018.03.22
4,우리 집은 맛나 된장 맛나,,2018.01.20,


### 2018년 2~3분기 방영한 드라마
-  2018년 4월 1일부터 2018년 9월 30일 사이에 방영한 드라마

In [0]:
df_dra = df_dra[df_dra['End_date'] >= '2018.04.01']
df_dra = df_dra[df_dra['Link'] != '']

## Get Ratings Data from Wikipedia

### make list with links

In [0]:
li_dra = df_dra['Link'].tolist()

### Extract data from table on Wikipedia
- 시청률 데이터 추출

In [0]:
def makelist2(table):
  url_base = 'https://ko.wikipedia.org'
  result = []
  allrows = table.findAll('tr')
  for row in allrows:
    result.append([])
    allcols = row.findAll('td')
    if len(allcols) > 2:
      for col in allcols[-2:]:
        texts = col.text.replace('《','').replace('》','').replace(' ','').replace('\n','').replace('%','')
        result[-1].append(texts)
    elif len(allcols) == 2:
      for col in allcols[-1:]:
        texts = col.text.replace('《','').replace('》','').replace(' ','').replace('\n','').replace('%','')
        result[-1].append(texts)
  return result


### make rating scores
- 드라마별 평균 시청률 구하기

In [10]:
score_list = []
for li in tqdm(li_dra):
  exceptions = ['https://ko.wikipedia.org/wiki/%EB%AF%B8%EC%8A%A4%ED%84%B0_%EC%85%98%EC%83%A4%EC%9D%B8',
                'https://ko.wikipedia.org/wiki/%EB%9D%BC%EC%9D%B4%ED%94%84_(2018%EB%85%84_%EB%93%9C%EB%9D%BC%EB%A7%88)',
                'https://ko.wikipedia.org/wiki/%EB%82%B4_%EB%92%A4%EC%97%90_%ED%85%8C%EB%A6%AC%EC%9A%B0%EC%8A%A4',
                'https://ko.wikipedia.org/wiki/KBS_%EB%93%9C%EB%9D%BC%EB%A7%88_%EC%8A%A4%ED%8E%98%EC%85%9C#2018년']
  res = requests.get(li).text
  soup = bs(res, 'lxml')
  tables = soup.find_all('table',{'class':'wikitable'})
  if li in exceptions[:-1]:
    scores = [np.nanmean([float(x) for x in li if x]) for li in makelist2(tables[0]) if li]
    score = round(np.nanmean(scores),2)
  elif li == exceptions[-1]:
    score = 0
  else :
    if 1 < len(tables) <= 2: 
      scores = [np.nanmean([float(x) for x in li if x]) for li in makelist2(tables[0]) if li]
      score = round(np.nanmean(scores),2)
    elif len(tables) > 2:
      scores = [np.nanmean([float(x) for x in li if x]) for li in makelist2(tables[1]) if li]
      score = round(np.nanmean(scores),2)
    else:
      score = 0
  score_list.append(score)
  

/usr/local/lib/python3.6/dist-packages/tqdm/_tqdm_notebook.py:88: TqdmExperimentalWarning: Detect Google Colab 0.0.1a2 and thus load dummy ipywidgets package. Note that UI is different from that in Jupyter. See https://github.com/tqdm/tqdm/pull/640
  " See https://github.com/tqdm/tqdm/pull/640".format(colab.__version__), TqdmExperimentalWarning)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: RuntimeWarning: Mean of empty slice


### Put rating score in dataframe
 - 데이터셋에 시청률 정보 추가

In [11]:
df_dra['Score'] = score_list
df_dra.head()

,Title,Link,Start_date,End_date,Score
12,으라차차 와이키키,https://ko.wikipedia.org/wiki/%EC%9C%BC%EB%9D%...,2018.02.05,2018.04.17,2.01
13,파도야 파도야,https://ko.wikipedia.org/wiki/%ED%8C%8C%EB%8F%...,2018.02.12,2018.08.31,0.00
15,키스 먼저 할까요,https://ko.wikipedia.org/wiki/%ED%82%A4%EC%8A%...,2018.02.20,2018.04.24,10.05
16,인형의 집,https://ko.wikipedia.org/wiki/%EC%9D%B8%ED%98%...,2018.02.26,2018.07.20,13.23
17,추리의 여왕 2,https://ko.wikipedia.org/wiki/%EC%B6%94%EB%A6%...,2018.02.28,2018.04.19,6.07


In [0]:
df_dra = df_dra[df_dra['Score'] > 0].sort_values(by = ['Score'],ascending=False)

In [0]:
df_dra.to_csv('gdrive/My Drive/공모전/Lpoint/데이터셋 만들기/drama.csv')

## 드라마에 등장한 상품 가져오기
- *'네이버 TV 속 이 상품'* 서비스를 제공하는 팅글([링크](http://www.tingle.kr))에서 데이터 수집



In [0]:
keywords = df_dra[df_dra['Score'] > df_dra['Score'].quantile(0.5)]['Title'].tolist()

In [16]:
keywords[:10]

['같이 살래요',
 '내일도 맑음',
 '미스터 션샤인',
 '인형의 집',
 '끝까지 사랑',
 '우리가 만난 기적',
 '데릴남편 오작두',
 '키스 먼저 할까요',
 '서른이지만 열일곱입니다',
 '백일의 낭군님']

### 드라마 id 가져오기


In [138]:
dra_id = []
for keyword in tqdm(keywords):
  url = 'https://api.infomore.co.kr/tingle/web/0.9/programs_search.json.html?page=1&_=T151503502&keyword={}&limit=200&token=null&category=1%7C2%7C3%7C7%7C8%7C9%7C10&genre=2'.format(keyword)
  res = requests.get(url)
  items = res.json().get('searchPrograms')
  try:
    dra_id.append({
        'title': keyword,
        'id':[str(item.get('id')) for item in items if item.get('title') == keyword]
    })
  except TypeError:
    pass

In [0]:
df_id = pd.DataFrame(dra_id)

### 드라마 id를 통해 상품 id 가져오기
- 드라마 마다 300개의 ppl 상품 추출

In [153]:
dra_items = []
for key in tqdm(dra_id):
  if key['id'] != []:
    url = 'https://api.infomore.co.kr/tingle/web/0.9/items_program.json.html?page=1&_=T153252785&id={}&limit=300&category=1%7C8&order=1&token=null&disable='.format(key['id'][0])
    res = requests.get(url)
    items = res.json().get('programItems')
    dra_items.append({
            'title': key['title'],
            'id':[str(item.get('id')) for item in items]})

### 상품 id를 활용해 상품 정보 가져오기

In [156]:
item_lists2 = []
for dra in tqdm(dra_items[:]):
  item_list = []
  for key in tqdm(dra['id']):
    url = 'https://api.infomore.co.kr/tingle/web/0.9/item.json.html?id={}&location=web'.format(key)
    res = requests.get(url)
    info = res.json()
    item_list.append([info.get('brandName'), info.get('itemTypeName')])
  item_lists2.append({
      'title': dra['title'],
      'lists': item_list
  })

### 명품만 남기기
 - 상품들 중 명품에 해당하는 상품만 남기기

In [0]:
lotte = pd.read_csv("gdrive/My Drive/공모전/Lpoint/데이터셋 만들기/lotte.csv")
keys = lotte[['PD_BRA_NM','CLAC2_NM']].apply(lambda x: '{}_{}'.format(x[0],x[1]), axis=1)

In [160]:
full_items = []
for li in tqdm(item_lists):
  full_items.append([])
  full_items[-1].append(li['title'])
  full_items[-1].append(['_'.join(l).replace(' ','') for l in li['lists']])
  

### 브랜드 + ~~상품 분류~~ 별로 등장 수 count
 - 상품 분류를 구별해내기 어려움
  - 예) 가방(남성가방?/여성가방?), 셔츠(남성상의?/여성상의?)

In [0]:
drama_count = {}
for ti in tqdm(full_items):
  brands_count = {}
  for key in keys:
    brand = key.split('_')[0]
    
    if brand not in brands_count:
      brands_count[brand] = len([i for i in ti[1] if brand in i.split('_')[0]])
    else :
      brands_count[brand] += len([i for i in ti[1] if brand in i.split('_')[0]])
    drama_count[ti[0]] = brands_count


In [0]:
drama_count

In [192]:
df_bd = pd.DataFrame.from_dict(drama_count, orient='index').reset_index()
df_bd.rename({'index':'Title'},axis=1,inplace=True)
df_bd.head()

,Title,겐조,구찌,금강제화,내셔널지오그래픽,닥스,덱케,디올옴므,라코스테,레노마,...,라보라토리오올파티보,메르세데스벤츠,메종프란시스커정,몰튼브라운,바이레도,벤츠,아닉구딸,아틀리에코롱,크리드,펜할리곤스
0,같이 살래요,0,0,0,0,26,10,0,144,0,...,0,0,0,0,0,0,0,0,0,0
1,검법남녀,0,0,0,0,0,5,0,18,0,...,0,0,0,0,0,0,0,0,0,0
2,그녀로 말할 것 같으면,0,0,0,0,0,10,0,54,0,...,0,0,0,0,0,0,0,0,0,0
3,기름진 멜로,0,132,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,김비서가 왜 그럴까,0,11,0,0,0,5,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [301]:
df_d = pd.merge(df_dra,df_bd, on=['Title'], how = 'inner')
colnames = ['Link', 'Start_date', 'End_date']
df_d.drop(colnames, axis=1, inplace=True)
df_d.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26 entries, 0 to 25
Columns: 388 entries, Title to 펜할리곤스
dtypes: float64(1), int64(386), object(1)
memory usage: 79.0+ KB


In [304]:
df_d.head()

,Title,Score,겐조,구찌,금강제화,내셔널지오그래픽,닥스,덱케,디올옴므,라코스테,...,라보라토리오올파티보,메르세데스벤츠,메종프란시스커정,몰튼브라운,바이레도,벤츠,아닉구딸,아틀리에코롱,크리드,펜할리곤스
0,같이 살래요,28.16,0.0,0.0,0.0,0.0,732.16,281.60,0.0,4055.04,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,내일도 맑음,16.08,0.0,0.0,0.0,0.0,1254.24,0.00,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,미스터 션샤인,14.13,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,인형의 집,13.23,0.0,0.0,0.0,0.0,859.95,0.00,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,끝까지 사랑,11.69,0.0,0.0,0.0,0.0,303.94,58.45,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [0]:
df_d.iloc[:,2:] = df_d.iloc[:,2:].apply(lambda x: x*df_d['Score'])

In [0]:
def normalize(x):
  return round((x-np.mean(x))/(np.std(x)),2)

In [311]:
df_t = df_d.iloc[:,2:].apply(lambda x: sum(x), axis=0).reset_index()
df_t.columns = ['PD_BRA_NM','BRA_SC']
df_t['BRA_SC'] = normalize(df_t['BRA_SC'])
df_t.head()

,PD_BRA_NM,BRA_SC
0,겐조,-0.16
1,구찌,0.63
2,금강제화,-0.29
3,내셔널지오그래픽,-0.29
4,닥스,5.17


In [0]:
lotte_drama=pd.merge(lotte, df_t, on=['PD_BRA_NM'], how='inner')

In [0]:
lotte_drama.to_csv('gdrive/My Drive/공모전/Lpoint/데이터셋 만들기/lotte_drama.csv',index=False)